<a href="https://colab.research.google.com/github/IshtiSikder/Optimal-allocation-of-rescue-teams-for-hostage-situations/blob/updates/R_for_MDP_(NEW).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
xxxc# CTMC MDP model for rescue operational system!!!
# For steady state prob for optimal policy
prob <- function(opSet = policySet, t, lam, eps, mu) {
  opTRM = matrix(0, nrow = t+1, ncol = t+1)
  for (i in 2:t) {
    opTRM[i, i-1] = sum(i*eps, opSet[i]*mu)
    opTRM[i, i+1] = lam      
  }
  opTRM[1, 2] = lam
  opTRM[t+1, t] = sum(t*eps, opSet[t+1]*mu)
  
  k = t+1
  steady_prob <- matrix(0, nrow=1, ncol=k)
  state <- matrix(0, nrow=1, ncol=k-1)
  for (i in 1:(k-1)) {
    if (i-2 < 0) {
      state[i] <- (lam/opTRM[i+1,i])
    } else {
      state[i] <- state[i-1]*(lam/opTRM[i+1,i])
    }
  }
  
  steady_prob[1] <- (1+sum(state))^-1
  for (i in 2:k) {
    steady_prob[i] <- state[i-1]*steady_prob[1]
  }
  return(steady_prob)
}

# Check function to determine new policy is same with current or not 
checkPolicy = function(policySet, policySetnew) {
  n = 0
  for (i in policySet == policySetnew) {
    if (i == 'FALSE') {
      n = 1
    } 
  } 
  if (n == 0) {
    return("TRUE")
  } else {
    return("FALSE")
  }
} 

rewardR <- function(costR, RC, nr) {
  if (RC == "SR") {
    if (costR < 0) {
      result = -sqrt(-costR * nr)      
    } else {
      result = sqrt(costR * nr)
    }
  } else if (RC == "L") {
    result = costR * nr      
  } else {
    if (costR < 0) {
      result = -(costR * nr)^2      
    } else {
      result = (costR * nr)^2      
    }
  }
  return(result)
}

rewardT <- function(costT, TC, nt) {
  if (TC == "SR") {
    if (costT < 0) {
      result = -sqrt(-costT * nt)      
    } else {
      result = sqrt(costT * nt)
    }
  } else if (TC == "L") {
    result = costT * nt      
  } else {
    if (costT < 0) {
      result = -(costT * nt)^2      
    } else {
      result = (costT * nt)^2      
    }
  }
  return(result)
}

#########################################################
#EDITS: 22ND FEB,2023
#NEW SERVICE RATE STRUCTURE. ADD ORIGINAL ONE AS WELL
probR = function(nr, nt, eps, mu) {
  #if (nt == 0 & nr == 0) {
  if (nt == 0){
    result = 0
  } else {
    result = min(nr,nt)*mu / (min(nr,nt)*mu + nt*eps)
  }
  return(result)
}
#########################################################

# Define MDP function (for basic policy)
MDP = function(lam, mu, eps, numRescue, numTerror, costR, costT, RC, TC) {
  r = numRescue
  t = numTerror
  # Design the three dimension matrix(i,j,k)
  # k: Index of rescue team
  # i: state(current)
  # j: state(next)
  
  
  #########################################################
  #EDITS: 22ND FEB,2023
  #NEW SERVICE RATES STRUCTURE, ADD ORIGINAL ONE AS WELL
  aijMatrix = array(0, dim = c(t+1,t+1,r+1))
  for (k in 1:(r+1)) {
    for (i in 1:t) {
      aijMatrix[i,i+1,k] <- lam
      if (i > 1) {
        #if (k==4){
          #if(i==4){
            #cat('min(k-1,i-1)',min(k-1,i-1))}}
        aijMatrix[i,i-1,k] <- sum(min(k-1,i-1)*mu+(i-1)*eps)        
      }
    }
    aijMatrix[t+1,t,k] <- (min(k-1,t)*mu+t*eps)
    aijMatrix[1,2,k] <- lam
}

  for (k in 1:(r+1)) {
    for (i in 1:(t+1)) {
      
      aijMatrix[i,i,k] <- -sum(aijMatrix[i,,k])      
    }
  }
  #########################################################
  
  #print(aijMatrix[4,4,5])
  #cat('meu',mu)
  #cat('epsilon',eps)
  #break
  
  
  # Design the MDP algorithm
  # Value determination
  policy = array(0, dim = c(1,t+1)) 
  policyOld = array(0, dim = c(1,t+1))
  policyNew = array(2, dim=c(1,t+1)) # initially set all decision is 1 in first policy
  policyTrack = array(2, dim=c(1,t+1))
  iteration = 0
  gainMatrix = c()
  
  while (checkPolicy(policyOld, policyNew) == 'FALSE') {
    policyOld = policyNew
    policy = policyNew
    # Design the transition matrix regarding policy
    A = array(0, dim = c(t+1,t+1))
    for (i in 1:length(policy)) {
      A[i,] = aijMatrix[i,,(policy[i])]  
    }

    # Design the cost matrix regarding policy
    q = array(0, dim = c(t+1,1))
    for (i in 1:length(policy)) {
      cost = probR(nr=(policy[i]-1),nt=(i-1),eps,mu)*rewardR(costR, RC, nr=(policy[i]-1))+
        (1-probR(nr=(policy[i]-1),nt=(i-1),eps,mu))*rewardT(costT, TC, nt=(i-1))
      q[i,1] = cost
    }
    newA = cbind(array(1, dim=c(t+1,1)), -A[,1:t])
    solution = solve(newA) %*% q
    gain = solution[1] # First row of solution is gain value
    gainMatrix = append(gainMatrix, gain)
    
    # Policy improvement
    newsolution = array(0, dim=c(t+1,1))
    for (i in 1:t) {
      newsolution[i] = solution[i+1]
    }
    newsolution[t+1] = 0 # Set last v value as 0 to calcultate the test quantity in policy improvement
    improveMatrix = array(0, dim=c(t+1,r+1))
    for (i in 1:(t+1)) {
      for (j in 1:(r+1)) {
        # Cost when we have j rescue operation and i terrorists
        cost = probR(nr=(j-1),nt=(i-1),eps,mu)*rewardR(costR, RC, nr=(j-1))+
          (1-probR(nr=(j-1),nt=(i-1),eps,mu))*rewardT(costT, TC, nt=(i-1))
        improveMatrix[i,j] = cost + aijMatrix[i,,j] %*% newsolution
      }
    }  
    
    # pick the minimum value among the decision
    policyNew = array(1, dim = c(1,t+1))
    for (i in 1:length(policyNew)) {
      policyNew[i] = which.min(improveMatrix[i,])
    }
    iteration = iteration + 1
    policyTrack = rbind(policyTrack, policyNew)
    policyNew
  }
  for (i in 1:length(policyNew)) {
    policyNew[i] = policyNew[i] - 1
  }
  for (i in 1:nrow(policyTrack)) {
    for (j in 1:ncol(policyTrack)) {
      policyTrack[i,j] = policyTrack[i,j] - 1      
    }

  }

  


#   stateName = c()
#   for (i in 1:(t+1)) {
#     stateName = append(stateName, paste('terror',(i-1)))
#   }
#   Steady_prob = prob(opSet = policyNew, t, lam, eps, mu)  
#   colnames(policyTrack) = stateName
#   colnames(policyNew) = stateName
#   colnames(Steady_prob) = stateName
#   rownames(Steady_prob) = 'Probability'
#   rownames(policyNew) = 'Decision'
#   result <- list("Iteration" = iteration,
#                  "PolicyChange" = policyTrack,
#                  "Gain" = gainMatrix,
#                  "SteadyStateProb" = Steady_prob,
#                  "Optimal_policy" = policyNew)
  
   return(policyNew)
 
 }
# MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10, costR=1, costT=2, RC="L", TC="L")
# A = MDP(lam = 0.3546, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10, costR=1, costT=2, RC="L", TC="L")


 

# # 1 more iteration for example plot
# library(ggplot2)
# result = MDP(lam = 0.2473, mu = 0.2829, eps = 0.1718, numRescue=3, numTerror=5, costR=1, costT=2, RC="L", TC="L")
# result = result$Gain
# ddd = array(0, dim=c(5,2))
# for (i in 1:5) {
#   ddd[i,1] = i
#   ddd[i,2] = result[i]  
# }
# ddd[5,1] = 5
# ddd[5,2] = ddd[4,2]

# ddd = as.data.frame(ddd)
# ggplot(ddd, aes(x=V1, y=V2)) +
#   geom_line(size=1) +
#   labs(x = 'Number of iteration', y = 'Gain value',
#        title = 'Gain value change during iterations')


# # Experiment design
# rescueSet = c(1,2,3,4,5,6,7,8,9,10)
# gainResult = array(0, dim=c(5,length(rescueSet)))
# for (j in 1:5) {
#   for (i in 1:length(rescueSet)) {
#     val = rescueSet[i]
#     result = MDP(lam=0.2473, mu=0.2829, eps=0.1718, numRescue=val, numTerror=10, costR=1, costT=j, RC="L", TC="L")
#     gainResult[j,i] = tail(result$Gain, 1)
#   }  
# }
# gainResult = as.data.frame(t(gainResult))
# gainResult$index = rescueSet
# ggplot() +
#   geom_line(data=gainResult, aes(x=index, y=V1, color='black'), size=1) +
#   geom_line(data=gainResult, aes(x=index, y=V2, color='red'), size=1) +
#   geom_line(data=gainResult, aes(x=index, y=V3, color='blue'), size=1) +
#   geom_line(data=gainResult, aes(x=index, y=V4, color='purple'), size=1) +
#   geom_line(data=gainResult, aes(x=index, y=V5, color='green'), size=1) +
#   labs(x = 'Rescue forces capacity', y = 'Gain value of the system',
#        title = 'Gain of the system over rescue team capacity') +
#   scale_color_identity(name = "Terror cost",
#                        labels = c("cost=1", "cost=2", "cost=3", "cost=4", "cost=5"),
#                        breaks = c("black", "red", "blue", "purple", "green"),
#                        guide = "legend")

Cost equations are improved (15th May, 23)

In [86]:
# CTMC MDP model for rescue operational system!!!

# Check function to determine new policy is same with current or not 
checkPolicy = function(policySet, policySetnew) {
  n = 0
  for (i in policySet == policySetnew) {
    if (i == 'FALSE') {
      n = 1
    } 
  } 
  if (n == 0) {
    return("TRUE")
  } else {
    return("FALSE")
  }
} 


# Define MDP function (for basic policy)
MDP = function(lam, mu, eps, numRescue, numTerror, costT, RC, mult) {
  r = numRescue
  t = numTerror
  
  #calculating costR

  costR = mult*costT*(mu+eps)


  #########################################################
  #EDITS: 22ND FEB,2023
  #NEW SERVICE RATES STRUCTURE, ADD ORIGINAL ONE AS WELL
  aijMatrix = array(0, dim = c(t+1,t+1,r+1))
  for (k in 1:(r+1)) {
    for (i in 1:t) {
      aijMatrix[i,i+1,k] <- lam
      if (i > 1) {
        #if (k==4){
          #if(i==4){
            #cat('min(k-1,i-1)',min(k-1,i-1))}}
        aijMatrix[i,i-1,k] <- sum(min(k-1,i-1)*mu+(i-1)*eps)        
      }
    }
    aijMatrix[t+1,t,k] <- (min(k-1,t)*mu+t*eps)
    aijMatrix[1,2,k] <- lam
}

  for (k in 1:(r+1)) {
    for (i in 1:(t+1)) {
      
      aijMatrix[i,i,k] <- -sum(aijMatrix[i,,k])      
    }
  }
  #########################################################
  
  
  # Design the MDP algorithm
  # Value determination
  policy = array(0, dim = c(1,t+1)) 
  policyOld = array(0, dim = c(1,t+1))
  policyNew = array(2, dim=c(1,t+1)) # initially set all decision is 1 in first policy
  policyTrack = array(2, dim=c(1,t+1))
  iteration = 0
  gainMatrix = c()
  
  while (checkPolicy(policyOld, policyNew) == 'FALSE') {
    policyOld = policyNew
    policy = policyNew
    # Design the transition matrix regarding policy
    A = array(0, dim = c(t+1,t+1))
    for (i in 1:length(policy)) {
      A[i,] = aijMatrix[i,,(policy[i])]  
    }

    # Design the cost matrix regarding policy

    q = array(0,dim=c(t+1,1))
    for (i in 1:length(policy)) {

      if (RC == 'L'){
      cost = (policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'SR'){
      cost = sqrt(policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'Q'){
      cost = ((policy[i]-1)^2)*costR + (i-1)*eps*costT
      }

      q[i,1] = cost
    }


    newA = cbind(array(1, dim=c(t+1,1)), -A[,1:t])
    solution = solve(newA) %*% q
    gain = solution[1] # First row of solution is gain value
    gainMatrix = append(gainMatrix, gain)
    
    # Policy improvement
    newsolution = array(0, dim=c(t+1,1))
    for (i in 1:t) {
      newsolution[i] = solution[i+1]
    }
    newsolution[t+1] = 0 # Set last v value as 0 to calcultate the test quantity in policy improvement
    improveMatrix = array(0, dim=c(t+1,r+1))
    for (i in 1:(t+1)) {
      for (j in 1:(r+1)) {
        # Cost when we have j rescue operation and i terrorists

        if (RC == 'L'){
      cost = (j-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'SR'){
      cost = sqrt(j-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'Q'){
      cost = ((j-1)^2)*costR + (i-1)*eps*costT
      }

        improveMatrix[i,j] = cost + aijMatrix[i,,j] %*% newsolution

      }
    }  
    
    # pick the minimum value among the decision
    policyNew = array(1, dim = c(1,t+1))
    for (i in 1:length(policyNew)) {
      policyNew[i] = which.min(improveMatrix[i,])
    }
    iteration = iteration + 1
    policyTrack = rbind(policyTrack, policyNew)
    policyNew
  }
  for (i in 1:length(policyNew)) {
    policyNew[i] = policyNew[i] - 1
  }
  for (i in 1:nrow(policyTrack)) {
    for (j in 1:ncol(policyTrack)) {
      policyTrack[i,j] = policyTrack[i,j] - 1      
    }

  }

  
   return(list(policyNew,improveMatrix,aijMatrix,newsolution))
 
 }

In [21]:
rndm = array(0, dim=c(1,11))
result = MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10, costT=1,RC='L')

In [66]:
MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10, costT=1,RC='L',mult=0.54)

0,1,2,3,4,5,6,7,8,9,0
0.2535969,0.4991349,0.7446729,0.9902109,1.2357489,1.4812869,1.7268249,1.9723629,2.2179009,2.4634389,2.7089769
0.2677134,0.2535969,0.4991349,0.7446729,0.9902109,1.2357489,1.4812869,1.7268249,1.9723629,2.2179009,2.4634389
0.2818299,0.2677134,0.2535969,0.4991349,0.7446729,0.9902109,1.2357489,1.4812869,1.7268249,1.9723629,2.2179009
0.2959465,0.2818299,0.2677134,0.2535969,0.4991349,0.7446729,0.9902109,1.2357489,1.4812869,1.7268249,1.9723629
0.3100630,0.2959464,0.2818299,0.2677134,0.2535969,0.4991349,0.7446729,0.9902109,1.2357489,1.4812869,1.7268249
0.3241791,0.3100626,0.2959462,0.2818297,0.2677133,0.2535969,0.4991349,0.7446729,0.9902109,1.2357489,1.4812869
0.3382911,0.3241754,0.3100597,0.2959440,0.2818283,0.2677126,0.2535969,0.4991349,0.7446729,0.9902109,1.2357489
0.3523550,0.3382467,0.3241384,0.3100301,0.2959218,0.2818135,0.2677052,0.2535969,0.4991349,0.7446729,0.9902109
0.3657696,0.3517480,0.3377264,0.3237048,0.3096832,0.2956616,0.2816400,0.2676184,0.2535969,0.4991349,0.7446729
0.3693957,0.3565292,0.3436626,0.3307961,0.3179295,0.3050630,0.2921965,0.2793299,0.2664634,0.2535969,0.4991349


In [87]:
current_result = MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10, costT=1,RC='L',mult=2)[1]

In [91]:
for (i in current_result){
  print(i+10)
}

     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
[1,]   10   10   10   10   10   10   10   10   10    10    10


In [90]:
current_result

0,0,0,0,0,0,0,0,0,0,0


In [84]:
mult_test = function(cost_terror,rc,low_lim,up_lim,incr) {

prev_result = array(100,dim=c(1,11))

for (val in seq(low_lim,up_lim, by = incr)){

        current_result = MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10, costT=cost_terror, RC=rc, mult=val)[1]
        impMat = MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10, costT=cost_terror, RC=rc, mult=val)[2]
        


        
        if (!(identical(current_result,prev_result))) {

          cat('\n\n','mult:',val,',','RC:',rc,',','\n\n')
          cat('optimal policy:\n')
          print(current_result)

          cat('improveMatrix:\n\n')
          print(impMat)

          prev_result = current_result

        }

    }

}

Linear structure, different factor values

In [85]:
mult_test(1,'L',0,100,0.01)



 mult: 0 , RC: L , 

optimal policy:
[[1]]
     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
[1,]    0    1    2    3    4    5    6    7    8     9    10

improveMatrix:

[[1]]
           [,1]      [,2]      [,3]      [,4]      [,5]      [,6]      [,7]
 [1,] 0.1043949 0.1043949 0.1043949 0.1043949 0.1043949 0.1043949 0.1043949
 [2,] 0.2112834 0.1043949 0.1043949 0.1043949 0.1043949 0.1043949 0.1043949
 [3,] 0.3181719 0.2112834 0.1043949 0.1043949 0.1043949 0.1043949 0.1043949
 [4,] 0.4250605 0.3181719 0.2112834 0.1043949 0.1043949 0.1043949 0.1043949
 [5,] 0.5319490 0.4250605 0.3181719 0.2112834 0.1043949 0.1043949 0.1043949
 [6,] 0.6388374 0.5319489 0.4250604 0.3181719 0.2112834 0.1043949 0.1043949
 [7,] 0.7457243 0.6388361 0.5319478 0.4250596 0.3181714 0.2112831 0.1043949
 [8,] 0.8525944 0.7457088 0.6388231 0.5319375 0.4250518 0.3181662 0.2112805
 [9,] 0.9592367 0.8523815 0.7455262 0.6386710 0.5318158 0.4249606 0.3181053
[10,] 1.0624449 0.9559949 0.8495449 0.7430949 0.

Linear cost structure, factor value = 1

In [51]:
MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10,costT=1, RC='L',mult=1)

0,0,0,0,0,0,0,0,0,0,0


Square root cost structure, different factor values

In [55]:
mult_test(1,'SR',0,100,0.01)

mult: 0 , RC: SR , 

0 1 2 3 4 5 6 7 8 9 10 

mult: 0.5 , RC: SR , 

0 0 2 3 4 5 6 7 8 9 10 

mult: 0.78 , RC: SR , 

0 0 0 3 4 5 6 7 8 9 10 

mult: 1.01 , RC: SR , 

0 0 0 0 4 5 6 7 8 9 10 

mult: 1.2 , RC: SR , 

0 0 0 0 0 5 6 7 8 9 10 

mult: 1.36 , RC: SR , 

0 0 0 0 0 0 6 7 8 9 10 

mult: 1.5 , RC: SR , 

0 0 0 0 0 0 0 7 8 9 10 

mult: 1.63 , RC: SR , 

0 0 0 0 0 0 0 0 8 9 10 

mult: 1.66 , RC: SR , 

0 0 0 0 0 0 0 0 8 9 0 

mult: 1.75 , RC: SR , 

0 0 0 0 0 0 0 0 0 9 0 

mult: 1.82 , RC: SR , 

0 0 0 0 0 0 0 0 0 0 0 



Square root cost structure, factor value = 1

In [53]:
MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10,costT=1, RC='SR',mult=1)

0,0,0,3,4,5,6,7,8,9,10


Quadratic cost structure, different factor values

In [56]:
mult_test(1,'Q',0,100,0.01)

mult: 0 , RC: Q , 

0 1 2 3 4 5 6 7 8 9 10 

mult: 0.02 , RC: Q , 

0 1 2 3 4 5 6 7 8 9 9 

mult: 0.03 , RC: Q , 

0 1 2 3 4 5 6 6 6 7 6 

mult: 0.04 , RC: Q , 

0 1 2 3 4 5 5 5 5 5 5 

mult: 0.05 , RC: Q , 

0 1 2 3 4 4 4 4 5 5 4 

mult: 0.06 , RC: Q , 

0 1 2 3 3 4 4 4 4 4 4 

mult: 0.07 , RC: Q , 

0 1 2 3 3 3 3 3 3 4 3 

mult: 0.08 , RC: Q , 

0 1 2 3 3 3 3 3 3 3 3 

mult: 0.09 , RC: Q , 

0 1 2 2 3 3 3 3 3 3 3 

mult: 0.1 , RC: Q , 

0 1 2 2 2 2 3 3 3 3 2 

mult: 0.11 , RC: Q , 

0 1 2 2 2 2 2 2 2 2 2 

mult: 0.16 , RC: Q , 

0 1 1 2 2 2 2 2 2 2 2 

mult: 0.17 , RC: Q , 

0 1 1 1 2 2 2 2 2 2 1 

mult: 0.18 , RC: Q , 

0 1 1 1 1 2 2 2 2 2 1 

mult: 0.19 , RC: Q , 

0 1 1 1 1 1 1 2 2 2 1 

mult: 0.2 , RC: Q , 

0 1 1 1 1 1 1 1 1 1 1 

mult: 0.53 , RC: Q , 

0 1 1 1 1 1 1 1 1 1 0 

mult: 0.61 , RC: Q , 

0 1 1 1 1 1 1 1 1 0 0 

mult: 0.62 , RC: Q , 

0 1 1 1 1 1 1 1 0 0 0 

mult: 0.63 , RC: Q , 

0 0 0 0 0 0 0 0 0 0 0 



Quadratic cost structure, factor value = 1

In [57]:
MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10,costT=1, RC='Q',mult=1)

0,0,0,0,0,0,0,0,0,0,0


In [15]:
`for (rc in list("SR","L","Q")){
    for (cost in seq(1,200, by = 0.1)){
        result = MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10,costT=cost, RC=rc)
      
        val = 0
        for (i in 1:length(result)){
        if (result[i] != 0 ) {
            if (result[i] != 1 ) {
                  val = val + 1
                   break
                    }
                  }
                  }
      if (val == 0){
       
        cat('costT:',cost,',','RC:',rc,'\n\n')
        cat(result,'\n\n')
        break
      }
    }
  
  }

costT: 1 , RC: Q 

0 1 1 1 1 1 1 1 1 1 1 



In [ ]:
#SINGLE TEST
#costR is constant, costT increases 

rc = 'L'
tc = 'L'

for (cost in seq(1,20, by = 0.1)){
  #for (rc in list("SR","L","QD")){
    #for (tc in list("SR","L","QD")){
        result = MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=10, numTerror=10, costR=cost, costT=1, RC=rc, TC=tc)
        cat('costR:',cost,',','costT:',1,',','RC:',rc,',','TC:',tc,'\n\n')
        cat(result,'\n\n')
    }
  #}
  #}



costR: 1 , costT: 1 , RC: L , TC: L 

0 1 2 3 4 5 6 7 8 9 9 

costR: 1.1 , costT: 1 , RC: L , TC: L 

0 1 2 3 4 5 5 6 7 8 8 

costR: 1.2 , costT: 1 , RC: L , TC: L 

0 1 2 3 3 4 5 6 7 7 8 

costR: 1.3 , costT: 1 , RC: L , TC: L 

0 1 2 2 3 4 5 5 6 7 7 

costR: 1.4 , costT: 1 , RC: L , TC: L 

0 1 2 2 3 4 4 5 6 6 7 

costR: 1.5 , costT: 1 , RC: L , TC: L 

0 1 1 2 3 3 4 5 5 6 6 

costR: 1.6 , costT: 1 , RC: L , TC: L 

0 1 1 2 3 3 4 4 5 6 6 

costR: 1.7 , costT: 1 , RC: L , TC: L 

0 1 1 2 2 3 4 4 5 5 5 

costR: 1.8 , costT: 1 , RC: L , TC: L 

0 1 1 2 2 3 3 4 4 5 5 

costR: 1.9 , costT: 1 , RC: L , TC: L 

0 1 1 2 2 3 3 4 4 5 5 

costR: 2 , costT: 1 , RC: L , TC: L 

0 1 1 2 2 3 3 4 4 4 5 

costR: 2.1 , costT: 1 , RC: L , TC: L 

0 1 1 1 2 2 3 3 4 4 4 

costR: 2.2 , costT: 1 , RC: L , TC: L 

0 1 1 1 2 2 3 3 4 4 4 

costR: 2.3 , costT: 1 , RC: L , TC: L 

0 1 1 1 2 2 3 3 3 4 4 

costR: 2.4 , costT: 1 , RC: L , TC: L 

0 1 1 1 2 2 3 3 3 4 4 

costR: 2.5 , costT: 1 , RC: L , TC: L 

0 1 